In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import torchvision
from PIL import Image

os.chdir('/content/drive/MyDrive/AIdata/Leaf Disease Dataset')

class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self, phase='train'):
   # 데이터 읽기
   self.dinfo = torchvision.datasets.ImageFolder(root='train')
   self.data_transfroms=transforms.Compose([transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  def __len__(self):
    return len(self.dinfo.samples)

  def __getitem__(self, idx): 
    x = Image.open(self.dinfo.imgs[idx][0])
    x = self.data_transfroms(x)
    y = torch.tensor(self.dinfo.imgs[idx][1])
    return x, y


class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 256, 256, 1)
    
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 4 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 4, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

batch_=100
dataloader ={ 'train' :  DataLoader(CustomDataset(phase='train'), batch_size=batch_, shuffle=True),
              # 'val' :  DataLoader(CustomDataset(phase='val'), batch_size=batch_, shuffle=False)
 }

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = CNN().to(device) # 모델 구축
loss_fn =  torch.nn.CrossEntropyLoss()  # 손실함수 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 옵티마이저 매개변수 갱신

nb_epochs = 10
for epoch in range(0, nb_epochs):
  avg_cost = 0
  sample_size = 0
  avg_cost_ = 0
  sample_size_ = 0

  # Train
  for batch_idx, samples in enumerate(dataloader['train']):
    x_train, y_train = samples
    x_train = x_train.to(device)
    y_train = y_train.to(device)

    pred = model(x_train)
    loss = loss_fn(pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avg_cost+=loss
    sample_size+=1

  print("epoch :" , epoch, "loss_train", avg_cost/sample_size )